In [1]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from time import sleep

import pickle
import os

from pprint import pprint

from hab.airtable.airtable import AirtableHAB, airtable_oauth
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

In [2]:
%%time

os.chdir("json_searches/")

dates = list()

df = pd.DataFrame()

for file in os.listdir():
    
    with open(file = file, mode = "br") as file:
        data = pickle.load(file)
        
    date = datetime.strptime(data["search_metadata"]["created_at"], "%Y-%m-%d %H:%M:%S UTC").date()
        
    try:
        
        df_ = pd.json_normalize(data["jobs_results"])
        
        for i in range(df_.shape[0]):
            
            dates.append(date)
            
        df = pd.concat([df, df_], ignore_index = True)
            
    except:
        pass
    

print(df.shape, len(dates))

os.chdir("..")

(40897, 12) 40897
CPU times: total: 5.3 s
Wall time: 1min 2s


In [6]:
dates

[datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023, 5, 4),
 datetime.date(2023,

In [3]:
%%time

# Creates "created_date"
df["date_posted"] = dates

# Drops Duplicates
df = df.drop_duplicates("job_id").reset_index(drop = True) 

# Cleans "via"
df["via"] = df["via"].apply(lambda x : clean.clean_source(x))

# Cleans "location"
df["location"] = df["location"].apply(lambda x : clean.clean_location(x))

# Cleans "contract_type"
df["detected_extensions.schedule_type"] = df["detected_extensions.schedule_type"].apply(lambda x : clean.clean_contract_type(x))

# Cleans "created_date"
df["detected_extensions.posted_at"] = df["detected_extensions.posted_at"].apply(lambda x : clean.transform_date(x))

df["date_posted"] = [clean.get_date(x, y) for x, y in df[["date_posted", "detected_extensions.posted_at"]].values]

# Creates "tech_skills"
df["tech_skills"] = df["description"].apply(lambda x : clean.get_skills(x))

CPU times: total: 7.69 s
Wall time: 10.2 s


In [4]:
df = clean.clean_column_names(df)

In [5]:
df

,title,company_name,location,source,description,job_highlights,related_links,thumbnail,extensions,job_id,detected_extensions.posted_at,contract_type,date_posted,tech_skills
0,Principal Data Scientist,Cabify,España,LinkedIn,"Do you want to change the world? At Cabify, th...",[{'items': ['Do you want to change the world? ...,"[{'link': 'http://cabify.com/', 'text': 'cabif...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 3 horas, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQcmluY2lwYWwgRGF0YSBTY2llbn...,0 days 03:00:00,Full-time,2023-05-04,"[Excel, Python, SQL]"
1,Data Scientist (d/f/m) – Data Analytics,"TK ELEVATOR IBERICA HOLDING, SL UNIPERSONAL",Madrid,TK Elevator,People are at the center of everything we do a...,[{'items': ['People are at the center of every...,[{'link': 'https://www.google.es/search?gl=es&...,NaN,"[hace 2 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAoZC9mL2...,2 days 00:00:00,Full-time,2023-05-02,"[Pandas, Power BI, Python, Spark, SQL, Tableau]"
2,Data Scientist - HEALTH,Capgemini Engineering,Madrid,LinkedIn,The Role\n\nDo you want to work in one of the ...,[{'items': ['The Role Do you want to work in ...,"[{'link': 'http://www.altran.com/', 'text': 'a...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAtIEhFQU...,1 days 00:00:00,Full-time,2023-05-03,"[AWS, Azure, Excel]"
3,Data Science,Scalian Spain,España,BeBee,Data Science\n\nEspaña\n\n22 febrero 2023\n\nC...,[{'items': ['Data Science España 22 febrero ...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UiLCJodGlkb2...,1 days 00:00:00,Full-time,2023-05-03,"[Estadística, Machine Learning, Python, Spark,..."
4,Postdoctoral Fellow/Lab Manager for Genome Dat...,IRB Barcelona,Barcelona,IRB Barcelona,Created in 2005 by the Generalitat de Cataluny...,[{'items': ['Created in 2005 by the Generalita...,"[{'link': 'https://www.irbbarcelona.org/', 'te...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQb3N0ZG9jdG9yYWwgRmVsbG93L0...,1 days 00:00:00,Full-time,2023-05-03,[Excel]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29570,Técnico de Soporte Remoto,TRS-Spain,Madrid,TRS Staffing Solutions,﻿Soporte Remoto\nCFGM Informática (SMX) (Grado...,[{'items': ['﻿Soporte Remoto CFGM Informática ...,[{'link': 'https://www.google.es/search?hl=es&...,NaN,"[hace 15 horas, Contratista]",eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBkZSBTb3BvcnRlIF...,0 days 15:00:00,Contractor,2023-05-15,[]
29571,Tecnico de soporte helpdesk ingles 100 remoto,serviguidebpo,Madrid,Jobrapido.com,horario l a v de 8 a 17 o de 9 a 18 con flexib...,[{'items': ['horario l a v de 8 a 17 o de 9 a ...,[{'link': 'https://www.google.es/search?hl=es&...,NaN,"[hace 3 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgaG...,3 days 00:00:00,Full-time,2023-05-12,[]
29572,TÉCNICO/A DE SOPORTE ATENCION AL CLIENTE ( 100...,Ibermatica,Madrid,Es.talent.com,"Como parte de Ayesa, conformamos uno de los pr...","[{'items': ['Como parte de Ayesa, conformamos ...","[{'link': 'http://www.ibermatica.com/', 'text'...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 2 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJUw4lDTklDTy9BIERFIFNPUE9SVE...,2 days 00:00:00,Full-time,2023-05-13,[]
29573,Scrum master senior producto tecnico remoto,between,España,Jobrapido.com,desarrollar las funcionalidades que den soport...,[{'items': ['desarrollar las funcionalidades q...,[{'link': 'https://www.google.es/search?hl=es&...,NaN,"[hace 3 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJTY3J1bSBtYXN0ZXIgc2VuaW9yIH...,3 days 00:00:00,Full-time,2023-05-12,[]


## Salario

In [6]:
def extract_salary(string):
    
    string = string.split()
    
    string = [s for s in string if any([x for x in s if x.isnumeric()])]
    
#     string = [x if len(x) > 0 else np.nan for x in string]
    
    return string

In [7]:
df["numeros"] = df["description"].apply(lambda x : extract_salary(x))

df["numeros"] = [x if len(x) > 0 else np.nan for x in df["numeros"]]

In [8]:
for x, y in df.sample(10)[["numeros", "description"]].values:
    print(x)
    print(y)
    print("*"*50)

['20', '2', '53']
Cas Training empresa de referencia con más de 20 años en consultoría tecnológica outsourcing y formación especializada selecciona a profesionales de Ingeniero/a Cloud Redes y Seguridad para un importante proyecto. EXPERIENCIA al menos 2 años TITULACION Informática Matemáticas Ingeniero cloud comms y seguridad Ingeniero cloud Arquitecto y hands on en la parte de networking y seguridad de AWS/AZure/GCP. Experto en application networking con herramientas nativas de cloud (ELB AppGW DNS Route 53 Azure DNS API Gateway etc) Conocimientos en service mesh y kubernetes Conociminetos y hands on en automatización de infraestructura para CI/CD de soluciones de networking Expericiencia en lenguajes como Ansible python y Terrafiorm Definir arquitecturas de comunicaciones y seguridad en cloud Troubleshooting de las comunicaciones cloud e hibridacion Implementación por código de elementos apllication networking Conocimientos DNS NTP IPAM Avanzado Cloud Automation Avanzado Obligatorio

In [9]:
# $18.000
# $24.000

# bruto:18000-incentivos
# 18-25.000
# 500,00€
# 40-50k

In [10]:
df_salario = pd.DataFrame()

df_copy = df.copy()
df_copy["description"] = df_copy["description"].apply(lambda x : x.lower().replace(",", ""))

for key in ["salario", "salary", "€", "$", "K", ".000", "brutos", "netos", "anual", "bruto", "neto",
            "al año", "mensual", "hora", "eur", "usd", "euros", "dolars", "per"]:
    
    df_salario = pd.concat([df_salario, df_copy[df_copy["description"].str.contains(key)]])
    
df_salario = df_salario.drop_duplicates("job_id").reset_index(drop = True)

df_salario = df_salario.dropna(subset = "numeros")

In [11]:
# for x, y in df_salario.sample(10)[["numeros", "description"]].values:
#     print(x)
#     print(y)
#     print("*"*80)

In [12]:
df_salario.shape

(25646, 15)

In [13]:
# for x in posibilidades:
#     print(x)

In [14]:
# for x in posibilidades:
#     if "k" in x and len(x) < 20:
#         print(x)

## Formatos de Salarios

In [15]:
## Formatos: "_K - _K", "_ - _K" (Upper and Lower)

import re

def formato_1(text):
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*[Kk]-\d{1,3}(?:[\.,]\d{3})*)[Kk]\b"  # Matches salary information in the format "30-40K"
    matches = re.findall(pattern, text)
    return matches

## Formato: "_K"
def formato_2(text):
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*K)\b"  # Matches salary information in the format "30K"
    matches = re.findall(pattern, text)
    return matches

# (30K-40K), [30K-40], [30-40K]
def formato_3(text):
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*[Kk]?-\d{1,3}(?:[\.,]\d{3})*[Kk]?)\b"  # Matches salary information in formats like "30K-40K", "[30K-40]", or "[30-40K]"
    matches = re.findall(pattern, text)
    return matches

# 300 - 400 €, 300-400€
def formato_4(text):
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*\s*-\s*\d{1,3}(?:[\.,]\d{3})*\s*€)\b"  # Matches salary information in formats like "300 - 400 €" or "300-400€"
    matches = re.findall(pattern, text)
    return matches

In [16]:
def clean_k_1000(string1, string2):
    
    if ("k" in string1+string2) or ("K" in string1+string2) or (".000" in string1+string2):
        
        string1 = float(string1.strip("kK.000"))
        string2 = float(string2.strip("kK.000"))
        
        return string1*1000, string2*1000
    
    else:
        return float(string1), float(string2)

In [17]:
def all_formatos(string):
    
    resultados = list()
    
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*[Kk]-\d{1,3}(?:[\.,]\d{3})*)[Kk]\b"  # Matches salary information in the format "30-40K"
    matches = re.findall(pattern, string)
    
    resultados.extend(matches)
    
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*K)\b"  # Matches salary information in the format "30K"
    matches = re.findall(pattern, string)
    
    resultados.extend(matches)
    
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*[Kk]?-\d{1,3}(?:[\.,]\d{3})*[Kk]?)\b"  # Matches salary information in formats like "30K-40K", "[30K-40]", or "[30-40K]"
    matches = re.findall(pattern, string)
    
    resultados.extend(matches)
    
    pattern = r"\b(\d{1,3}(?:[\.,]\d{3})*\s*-\s*\d{1,3}(?:[\.,]\d{3})*\s*€)\b"  # Matches salary information in formats like "300 - 400 €" or "300-400€"
    matches = re.findall(pattern, string)
    
    resultados.extend(matches)
    
    resultados = list(set([x for x in resultados if (len(x) > 4) and (not x.startswith("00"))]))
    
    mitad = [x.split("-") for x in resultados]
    
    mitad = list(set([clean_k_1000(x, y) for x, y in mitad]))
    
    mitad = np.array([[x, y] for x, y in mitad if x < y]).flatten()
    
#     mitad = list([[clean_k_1000(x, y)] for x, y in mitad])

    mitad = [x for x in mitad if x > 100]
    
    return mitad if len(mitad) > 0 else np.nan

In [18]:
df_salario["prueba"] = df_salario["description"].apply(lambda x : all_formatos(x))

In [19]:
df = pd.concat([df, df_salario["prueba"]], axis = 1)

In [20]:
df["prueba"].dropna()

1         [3000.0, 45000.0]
2         [3000.0, 35000.0]
3          [3000.0, 4000.0]
5          [4000.0, 6000.0]
6          [3000.0, 4000.0]
                ...        
28620     [3000.0, 36000.0]
28845    [25000.0, 35000.0]
29228     [3000.0, 36000.0]
29261    [25000.0, 35000.0]
29430     [3000.0, 36000.0]
Name: prueba, Length: 487, dtype: object

In [21]:
print(df_salario.loc[3, :]["description"])

hasta el 15/12/funciones perfil data science con r (python) y sql desarrollar y mantener algoritmos para evaluación de riesgos automatización de decisiones y mejora continua de las herramientas de inteligencia de negocio implementación controlada y medida de nuevos desarrollos en formato sprint/scrum interacción con todos los departamentos de la empresa establecer procesos para el monitoramiento y mantenimiento de la creciente librería de algoritmos. denir estrategias de desarrollo y protección intelectual 1-3 años de experiência profesional en estadística aplicada/minería de datos/analítica avanzada análisis exploratorio de datos. conocimiento y experiência excelente en técnicas y algoritmos estadísticos para la completa caracterización de bloques de datos; regresiones lineales no lineales logísticas bayesianas iv woe entre otros. excelente conocimientos de las principales herramientas analíticas de data science; r y/o python conocimientos básicos para la manipulación de bases... de d

In [22]:
posibilidades = df_salario["description"].apply(lambda x : formato_1(x)).value_counts().index.to_list() +\
df_salario["description"].apply(lambda x : formato_2(x)).value_counts().index.to_list() +\
df_salario["description"].apply(lambda x : formato_3(x)).value_counts().index.to_list() +\
df_salario["description"].apply(lambda x : formato_4(x)).value_counts().index.to_list()

In [23]:
p = list()

for x in posibilidades:
    
    if len(x) > 1:
        for y in x:
        
            p.append(y)
    else:
        p.extend(x)
        
p = sorted(set(p))

p

['0-1',
 '0-2',
 '0-20',
 '0-25',
 '0-3',
 '0-6',
 '0-8',
 '00-06',
 '00-09',
 '00-10',
 '00-14',
 '00-15',
 '00-16',
 '00-17',
 '00-18',
 '00-19',
 '00-20',
 '00-22',
 '00-9',
 '000-60',
 '01-24',
 '02-16',
 '03-05',
 '03-30',
 '04-04',
 '04-18',
 '04-19',
 '04-24',
 '05-05',
 '05-16',
 '05-18',
 '07-04',
 '07-10',
 '074-023',
 '08-03',
 '09-02',
 '09-14',
 '1-1',
 '1-10',
 '1-2',
 '1-3',
 '1-4',
 '1-5',
 '10-15',
 '10-20',
 '10-30',
 '100-200',
 '11-12',
 '11-14',
 '11-17',
 '11-23',
 '11-50',
 '11-9',
 '12-04',
 '12-05',
 '12-14',
 '12-15',
 '12.000-18.000',
 '15-04',
 '15-12',
 '15-16',
 '15-18',
 '15-20',
 '15-30',
 '16-03',
 '16-12',
 '16-18',
 '17-18',
 '17-21k',
 '17.000-20.000',
 '175-24k',
 '18-20',
 '18-23k',
 '18-24',
 '18-24k',
 '18-25.000',
 '18.000-21.000',
 '18.000-24.000',
 '180-200',
 '18k-21',
 '18k-21k',
 '19-20k',
 '19.500-25',
 '1k-3',
 '1k-3k',
 '2-12',
 '2-3',
 '2-4',
 '2-5',
 '2-6',
 '2-8',
 '20-100',
 '20-23k',
 '20-25',
 '20-25k',
 '20-28.000',
 '20-30k',
 '2

In [24]:
finales = list()

for x in p:
    
    if "k" in x:
        
        finales.append(x)
        
finales

['17-21k',
 '175-24k',
 '18-23k',
 '18-24k',
 '18k-21',
 '18k-21k',
 '19-20k',
 '1k-3',
 '1k-3k',
 '20-23k',
 '20-25k',
 '20-30k',
 '20k-26',
 '20k-26k',
 '20k-28',
 '20k-28k',
 '20k-30',
 '20k-30k',
 '21-23k',
 '21-25k',
 '21-40k',
 '21k-24',
 '21k-24k',
 '21k-32',
 '21k-32k',
 '21k-35',
 '21k-35k',
 '22-32k',
 '22-33k',
 '22-36k',
 '23-27k',
 '23k-25',
 '23k-25k',
 '24-25k',
 '24-32k',
 '24-36k',
 '24-40k',
 '24k-27',
 '24k-27k',
 '24k-29',
 '24k-29k',
 '25-28k',
 '25-30k',
 '25-32k',
 '25-33k',
 '25-35k',
 '25-36k',
 '25k-29',
 '25k-29k',
 '25k-30',
 '25k-30k',
 '26-28k',
 '26-33k',
 '26-34k',
 '26k-30',
 '26k-30k',
 '26k-40',
 '26k-40k',
 '27-32k',
 '27-35k',
 '27-36k',
 '27k-30',
 '27k-30k',
 '27k-32',
 '27k-32k',
 '28-30k',
 '28-32k',
 '28-33k',
 '28-35k',
 '28-38k',
 '28-40k',
 '28-45k',
 '28.000-33.000k',
 '28k-32',
 '28k-32k',
 '30-35k',
 '30-36k',
 '30-38k',
 '30-40k',
 '30-43k',
 '30-45k',
 '30-47k',
 '30-48k',
 '30-65k',
 '30k-35',
 '30k-35k',
 '30k-36',
 '30k-36k',
 '30k-4

In [25]:
[[x.split("-")[0], x.split("-")[1]] for x in finales]

[['17', '21k'],
 ['175', '24k'],
 ['18', '23k'],
 ['18', '24k'],
 ['18k', '21'],
 ['18k', '21k'],
 ['19', '20k'],
 ['1k', '3'],
 ['1k', '3k'],
 ['20', '23k'],
 ['20', '25k'],
 ['20', '30k'],
 ['20k', '26'],
 ['20k', '26k'],
 ['20k', '28'],
 ['20k', '28k'],
 ['20k', '30'],
 ['20k', '30k'],
 ['21', '23k'],
 ['21', '25k'],
 ['21', '40k'],
 ['21k', '24'],
 ['21k', '24k'],
 ['21k', '32'],
 ['21k', '32k'],
 ['21k', '35'],
 ['21k', '35k'],
 ['22', '32k'],
 ['22', '33k'],
 ['22', '36k'],
 ['23', '27k'],
 ['23k', '25'],
 ['23k', '25k'],
 ['24', '25k'],
 ['24', '32k'],
 ['24', '36k'],
 ['24', '40k'],
 ['24k', '27'],
 ['24k', '27k'],
 ['24k', '29'],
 ['24k', '29k'],
 ['25', '28k'],
 ['25', '30k'],
 ['25', '32k'],
 ['25', '33k'],
 ['25', '35k'],
 ['25', '36k'],
 ['25k', '29'],
 ['25k', '29k'],
 ['25k', '30'],
 ['25k', '30k'],
 ['26', '28k'],
 ['26', '33k'],
 ['26', '34k'],
 ['26k', '30'],
 ['26k', '30k'],
 ['26k', '40'],
 ['26k', '40k'],
 ['27', '32k'],
 ['27', '35k'],
 ['27', '36k'],
 ['27k', '30

In [26]:
print(df_salario[df_salario["description"].str.contains("60-40")]["description"][1704])

en exportadora data base contamos con más de 20 años de experiencia en servicios de consultoría it. trabajamos siempre a la vanguardia en el campo de la innovación y la tecnología; tanto en la ingeniería de software como en la consultoría tecnológica.

actualmente buscamos un perfil de ingeniero/a informático de redes y seguridad para trabajar en un importante cliente.

requerido:
• conocimiento y experiencia de 6 años en:

o seguridad perimetral (firewall ips/ids). tecnología fortinet y palo alto.

o tecnologías de equilibrado de equipos. módulos apm y ltm de tecnología f5.

o seguridad en la navegación por internet. proxy de navegación.

o gestión de redes de comunicaciones (direccionamiento dimensionamiento enrutamiento etc.) tecnologías microsoft o infoblox

o gestión de grandes entornos con varios centros de proceso de datos

o gestión de proyectos de comunicación

y lo que es más :

o experiencia en redes de centros de datos de 4 años

o redes data center cisco aci... experiencia

In [27]:
# salarios = list()

# for x in df["extensions"]:
    

#     salarios.append([y.replace("\xa0", "") for y in x if "\xa0" in y])
    
# salarios = [x[0] if len(x) > 0 else np.nan for x in salarios]

# df["salaries"] = salarios

In [28]:
# print(df[df["salaries"] == "33.280€ por hora"]["description"][9032])

In [29]:
# map_clean_salary = {"hora" : 8*4*7*12,
#                     "día"  : 4*7*12,
#                     "semana" : 4*12,
#                     "mes"  : 12,
#                     "año"  : 1}

In [30]:
# def clean_salary(string):
    
#     if type(string) != str:
        
#         return np.nan
    
    
#     datos = list()
    
    
    
    
    
    
# #     string = string.replace("mil", ".000")
    
    
# #     if " a " in string:
        
# #         string = string.split(" a ")
        
# #     else:
# #         pass
        
        
#     return string

In [31]:
# salarios[1342]

In [32]:
# for x in df["salaries"].apply(lambda x : clean_salary(x)).value_counts().index:

#     print(x)

In [33]:
# set([x.split()[-1] for x in df["salaries"].apply(lambda x : clean_salary(x)).value_counts().index])

In [34]:
hab = AirtableHAB(**airtable_oauth)

df = hab.fill_na(df)

In [ ]:
%%time
df2 = hab.airtable_get()

In [ ]:
df2

In [37]:
df

,fldYT0MGMPE4crwM8,fldcFDIakt1GYcGZd,fldBITI7ca8oMiQ0e,fldvIt9No3oKkP9QN,fldULNjUV3l8uz5SV,job_highlights,related_links,thumbnail,extensions,fldK0dPjzrYTLbq8H,detected_extensions.posted_at,fldJygkANaRLHFENB,fldgPkal466rDXzMy,fldO98rLB9M3kQTnf,numeros,prueba
0,Principal Data Scientist,Cabify,España,LinkedIn,"Do you want to change the world? At Cabify, th...",[{'items': ['Do you want to change the world? ...,"[{'link': 'http://cabify.com/', 'text': 'cabif...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 3 horas, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQcmluY2lwYWwgRGF0YSBTY2llbn...,0 days 03:00:00,Full-time,2023-05-04,"[Excel, Python, SQL]","[11, 40, 8, L5, [55-85K€], 3rd, 1-3]",
1,Data Scientist (d/f/m) – Data Analytics,"TK ELEVATOR IBERICA HOLDING, SL UNIPERSONAL",Madrid,TK Elevator,People are at the center of everything we do a...,[{'items': ['People are at the center of every...,[{'link': 'https://www.google.es/search?gl=es&...,,"[hace 2 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAoZC9mL2...,2 days 00:00:00,Full-time,2023-05-02,"[Pandas, Power BI, Python, Spark, SQL, Tableau]",[3+],"[3000.0, 45000.0]"
2,Data Scientist - HEALTH,Capgemini Engineering,Madrid,LinkedIn,The Role\n\nDo you want to work in one of the ...,[{'items': ['The Role Do you want to work in ...,"[{'link': 'http://www.altran.com/', 'text': 'a...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAtIEhFQU...,1 days 00:00:00,Full-time,2023-05-03,"[AWS, Azure, Excel]","[270,000, 50]","[3000.0, 35000.0]"
3,Data Science,Scalian Spain,España,BeBee,Data Science\n\nEspaña\n\n22 febrero 2023\n\nC...,[{'items': ['Data Science España 22 febrero ...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UiLCJodGlkb2...,1 days 00:00:00,Full-time,2023-05-03,"[Estadística, Machine Learning, Python, Spark,...","[22, 2023, 1]","[3000.0, 4000.0]"
4,Postdoctoral Fellow/Lab Manager for Genome Dat...,IRB Barcelona,Barcelona,IRB Barcelona,Created in 2005 by the Generalitat de Cataluny...,[{'items': ['Created in 2005 by the Generalita...,"[{'link': 'https://www.irbbarcelona.org/', 'te...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQb3N0ZG9jdG9yYWwgRmVsbG93L0...,1 days 00:00:00,Full-time,2023-05-03,[Excel],"[2005, 2011., 29, 450, 41, CRISPR/Cas12a, 2020...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29570,Técnico de Soporte Remoto,TRS-Spain,Madrid,TRS Staffing Solutions,﻿Soporte Remoto\nCFGM Informática (SMX) (Grado...,[{'items': ['﻿Soporte Remoto CFGM Informática ...,[{'link': 'https://www.google.es/search?hl=es&...,,"[hace 15 horas, Contratista]",eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBkZSBTb3BvcnRlIF...,0 days 15:00:00,Contractor,2023-05-15,[],"[W8.1, W10,, 31/12]",
29571,Tecnico de soporte helpdesk ingles 100 remoto,serviguidebpo,Madrid,Jobrapido.com,horario l a v de 8 a 17 o de 9 a 18 con flexib...,[{'items': ['horario l a v de 8 a 17 o de 9 a ...,[{'link': 'https://www.google.es/search?hl=es&...,,"[hace 3 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgaG...,3 days 00:00:00,Full-time,2023-05-12,[],"[8, 17, 9, 18]",
29572,TÉCNICO/A DE SOPORTE ATENCION AL CLIENTE ( 100...,Ibermatica,Madrid,Es.talent.com,"Como parte de Ayesa, conformamos uno de los pr...","[{'items': ['Como parte de Ayesa, conformamos ...","[{'link': 'http://www.ibermatica.com/', 'text'...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 2 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJUw4lDTklDTy9BIERFIFNPUE9SVE...,2 days 00:00:00,Full-time,2023-05-13,[],"[11.000, 20, 50, 2, 2]",
29573,Scrum master senior producto tecnico remoto,between,España,Jobrapido.com,desarrollar las funcionalidades que den soport...,[{'items': ['desarrollar las funcionalidades q...,[{'link': 'https://www.google.es/search?

In [38]:
# df = df[~df["fldK0dPjzrYTLbq8H"].isin(df2["job_id"])].reset_index(drop = True)

In [39]:
df = df[['fldYT0MGMPE4crwM8', 'fldcFDIakt1GYcGZd', 'fldBITI7ca8oMiQ0e',
    'fldvIt9No3oKkP9QN', 'fldULNjUV3l8uz5SV', 'fldK0dPjzrYTLbq8H',
    'fldJygkANaRLHFENB', 'fldgPkal466rDXzMy', 'fldO98rLB9M3kQTnf']]

df["fldgPkal466rDXzMy"] = df["fldgPkal466rDXzMy"].astype(str)

C:\Users\danie\AppData\Local\Temp\ipykernel_11540\957058650.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fldgPkal466rDXzMy"] = df["fldgPkal466rDXzMy"].astype(str)


In [40]:
df

,fldYT0MGMPE4crwM8,fldcFDIakt1GYcGZd,fldBITI7ca8oMiQ0e,fldvIt9No3oKkP9QN,fldULNjUV3l8uz5SV,fldK0dPjzrYTLbq8H,fldJygkANaRLHFENB,fldgPkal466rDXzMy,fldO98rLB9M3kQTnf
0,Principal Data Scientist,Cabify,España,LinkedIn,"Do you want to change the world? At Cabify, th...",eyJqb2JfdGl0bGUiOiJQcmluY2lwYWwgRGF0YSBTY2llbn...,Full-time,2023-05-04,"[Excel, Python, SQL]"
1,Data Scientist (d/f/m) – Data Analytics,"TK ELEVATOR IBERICA HOLDING, SL UNIPERSONAL",Madrid,TK Elevator,People are at the center of everything we do a...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAoZC9mL2...,Full-time,2023-05-02,"[Pandas, Power BI, Python, Spark, SQL, Tableau]"
2,Data Scientist - HEALTH,Capgemini Engineering,Madrid,LinkedIn,The Role\n\nDo you want to work in one of the ...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAtIEhFQU...,Full-time,2023-05-03,"[AWS, Azure, Excel]"
3,Data Science,Scalian Spain,España,BeBee,Data Science\n\nEspaña\n\n22 febrero 2023\n\nC...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UiLCJodGlkb2...,Full-time,2023-05-03,"[Estadística, Machine Learning, Python, Spark,..."
4,Postdoctoral Fellow/Lab Manager for Genome Dat...,IRB Barcelona,Barcelona,IRB Barcelona,Created in 2005 by the Generalitat de Cataluny...,eyJqb2JfdGl0bGUiOiJQb3N0ZG9jdG9yYWwgRmVsbG93L0...,Full-time,2023-05-03,[Excel]
...,...,...,...,...,...,...,...,...,...
29570,Técnico de Soporte Remoto,TRS-Spain,Madrid,TRS Staffing Solutions,﻿Soporte Remoto\nCFGM Informática (SMX) (Grado...,eyJqb2JfdGl0bGUiOiJUw6ljbmljbyBkZSBTb3BvcnRlIF...,Contractor,2023-05-15,[]
29571,Tecnico de soporte helpdesk ingles 100 remoto,serviguidebpo,Madrid,Jobrapido.com,horario l a v de 8 a 17 o de 9 a 18 con flexib...,eyJqb2JfdGl0bGUiOiJUZWNuaWNvIGRlIHNvcG9ydGUgaG...,Full-time,2023-05-12,[]
29572,TÉCNICO/A DE SOPORTE ATENCION AL CLIENTE ( 100...,Ibermatica,Madrid,Es.talent.com,"Como parte de Ayesa, conformamos uno de los pr...",eyJqb2JfdGl0bGUiOiJUw4lDTklDTy9BIERFIFNPUE9SVE...,Full-time,2023-05-13,[]
29573,Scrum master senior producto tecnico remoto,between,España,Jobrapido.com,desarrollar las funcionalidades que den soport...,eyJqb2JfdGl0bGUiOiJTY3J1bSBtYXN0ZXIgc2VuaW9yIH...,Full-time,2023-05-12,[]


In [42]:
hab.airtable_post(df)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
